# COVID-19_Analysis
### Generates standalone interactive HTML &/or deployment on web server (local host or cloud provider)

## Author: Prasanna Badami

### <font size = '4'> Note:
<font size = '4'> This source code,
- <font size = '3'> runs in Jupyter notebook with environment meeting all libraries used in this source code.
- <font size = '3'> generates standalone HTML output in the 'output' directory in the directory of this source code file. User can modify the code to generate the output in Jupyter notebook or to other directory.
- <font size = '3'> needs internet connection, it downloads three input files from CSSEGISandData repository on GitHub. Internet charges may apply.
- <font size = '3'> to calculate incident rate, by default downloads & process 'UID_ISO_FIPS_LookUp_Table.csv' from CSSEGISandData & converts into population record or it also has an option in the code to take hardcorded population record, which by default is disabled in the code.
    
  
<font size = '3'>To get the output in web browser similar to standalone interactive HTML without exposing code, you need to use Bokeh server deployment, which deploys output of this source code on local system or cloud provider. This is done by adding panel.servable() in source file & launching the app using 'panel serve sourcefile.ipynb' at command prompt 

In [ ]:
import pandas as pd
from datetime import datetime
import collections
import numpy as np

In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, Div, HoverTool, FuncTickFormatter, NumeralTickFormatter
from bokeh.resources import INLINE
from bokeh.layouts import gridplot
from bokeh.transform import factor_cmap

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import panel as pn
pn.extension()
import hvplot.pandas

In [ ]:
def set_external_graph_prop(p):
    p.yaxis.formatter = NumeralTickFormatter(format = '0.0a')
    p.y_range.start = 0
    p.toolbar_location = None
    p.margin = 5
    p.background_fill_color = 'white'
    p.border_fill_color = 'white'
    p.width = 300
    p.height = 300
    
    p.xaxis.axis_label = 'Date'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.xaxis.major_label_text_font_style = 'bold'
    p.xaxis.major_label_orientation = 45
    
    p.yaxis.axis_label = 'COVID19_Case'
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.major_label_text_font_style = 'bold'
#     p.xgrid.minor_grid_line_alpha = 0.1
#     p.ygrid.minor_grid_line_alpha = 0.1
#     p.xgrid.minor_grid_line_color = 'black'
#     p.ygrid.minor_grid_line_color = 'black'
#     p.yaxis.axis_label = args[0] + 'Cases'

In [ ]:
def set_external_graph_prop2(p):
    p.yaxis.formatter = FuncTickFormatter(code=""" return (Math.floor(tick)) + "%" """)
    p.y_range.start = 0
    p.toolbar_location = None
    p.margin = 5
    p.background_fill_color = 'white'
    p.border_fill_color = 'white'
    p.width = 300
    p.height = 300
    
    p.xaxis.axis_label = 'Date'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.xaxis.major_label_text_font_style = 'bold'
    p.xaxis.major_label_orientation = 45

    p.yaxis.axis_label = 'Case Fatality Ratio'
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.major_label_text_font_style = 'bold'

In [ ]:
def set_external_graph_prop3(p):
    p.yaxis.formatter = NumeralTickFormatter(format = '0.0a')
    p.y_range.start = 0
    p.toolbar_location = None
    p.margin = 5
    p.background_fill_color = 'white'
    p.border_fill_color = 'white'
    p.width = 300
    p.height = 300
    
    p.xaxis.axis_label = 'Date'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.xaxis.major_label_orientation = 45
    p.xaxis.major_label_text_font_style = 'bold'
    
    p.yaxis.axis_label = 'Incident Rate'
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.major_label_text_font_style = 'bold'

In [ ]:
def set_internal_graph_prop(p_GlyphRenderer):
    p_GlyphRenderer.glyph.line_width = 2
    p_GlyphRenderer.glyph.line_color = 'dodgerblue'

In [ ]:
todays_date = datetime.today().strftime('%a, %b %d, %Y')
# todays_date.replace('/', '-')

In [ ]:
# Function to create DataFrame out of .csv file
def CreateDF(df):
    df_prelude = df.drop(['Province/State', 'Lat', 'Long'], axis = 1).copy()
    df_prelude.columns = pd.to_datetime(df_prelude.columns)
    df_pivot = df_prelude.pivot_table(columns = df_prelude.index, aggfunc = 'sum')
  #  df_pivot.reset_index(inplace = True)
  #  df_pivot.rename(columns = {'index': 'Date'}, inplace = True)
    return(df_pivot)

In [ ]:
url_c = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_r  = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
url_d = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

In [ ]:
# Read & create a copy confirmed confirmed global cases
df_confirmed_main = CreateDF(pd.read_csv(url_c, index_col = ['Country/Region'], parse_dates  = True, infer_datetime_format= True))
df_confirmed = df_confirmed_main.copy()

In [ ]:
date_on_plot = df_confirmed.index[df_confirmed.index.size-1].strftime('%A, %b %d, %Y')
date_on_plot

In [ ]:
# Read & create a copy recovered global cases
df_recovered_main = CreateDF(pd.read_csv(url_r, index_col = ['Country/Region'], parse_dates  = True, infer_datetime_format= True))
df_recovered = df_recovered_main.copy()

In [ ]:
# Read & create a copy deaths global cases
df_deaths_main = CreateDF(pd.read_csv(url_d, index_col = ['Country/Region'], parse_dates  = True, infer_datetime_format= True))
df_deaths = df_deaths_main.copy()

In [ ]:
# DataFrames with worldwide column, these DataFrames are used in plots
df_confirmed_ww = df_confirmed.copy()
df_confirmed_ww['Worldwide'] = df_confirmed.sum(axis = 1)
df_recovered_ww = df_recovered.copy()
df_recovered_ww ['Worldwide'] = df_recovered.sum(axis = 1)
df_deaths_ww = df_deaths.copy()
df_deaths_ww['Worldwide'] = df_deaths.sum(axis = 1)

In [ ]:
# DataFrame for top 10 countries, this DataFrame is used in bar plot
# Top 10 recovery, deaths & active are with reference to confirmed cases
# Index of pd.Series df_confirmed_main.max().sort_values(ascending == False)[0:10] contains list of top 10 countries with max cases.
df_top10_crda = pd.DataFrame(dict(Confirmed = df_confirmed_main.max().sort_values(ascending = False)[0:10], 
                                 Recovered = df_recovered_main.max().sort_values(ascending = False)\
                                 [df_confirmed_main.max().sort_values(ascending = False)[0:10].index],
                                 Deaths = df_deaths_main.max().sort_values(ascending = False)\
                                 [df_confirmed_main.max().sort_values(ascending = False)[0:10].index]))
df_top10_crda['Active'] = df_top10_crda['Confirmed'] - df_top10_crda['Recovered'] - df_top10_crda['Deaths']

In [ ]:
# Country wise population is hardcoded in the source code here because there is less chance of it changing
# 'population_record' is the variable that stores countries & corresponding populations
# df_pop_inc_worldwide is generated using one of the two methods: 
    # 1. Using .csv file from repository, 
    # 2. Using hardcoded 'population_record'
# df_pop_inc_worldwide is used in calculation of incident_rate. 

# Method 1: Using .csv file from repository only if True:
population_record = list()  # Empty list, this will remain empty if False:
if True:
    url_population = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv'
    df_population = pd.read_csv(url_population)

#     df_population contain provinces/states for some countries, my interest is in country wise population.
#     So first filter 'Province_State' column for null/nan then filter corresponding 'Country_Region' & 'Population',
#     no need to use sum or pivot, 'Country_Region' column with country name has null/nan in 'Province_State' column and
#     corresponding 'Population' column field has sum of population of all provinces, if applicable. Please see the .csv file.   
    df_pop = df_population[df_population['Province_State'].isna()].filter(['Country_Region', 'Population'])

#     Create dataframe with single row for worldwide population
    df_pop_ww = pd.DataFrame({df_pop.columns[0] : 'Worldwide', df_pop.columns[1] : df_pop.sum(axis = 0)['Population']}, index = [0])

#     Final population dataframe that include worldwide population
    df_pop_inc_worldwide = pd.concat([df_pop_ww, df_pop], ignore_index = True)
#     Just to make population_record non-empty this is assigned here, you can assign any value
#     population_record is not used in calculation of incident_rate, df_pop_inc_worldwide is used.
    population_record = df_pop_inc_worldwide.to_dict(orient = 'records')

In [ ]:
# Method 2: Using hardcoded 'population_record' only if False:
if (population_record == []):
    population_record = [{'Country_Region': 'Worldwide', 'Population': 7711863221.0},
     {'Country_Region': 'Afghanistan', 'Population': 38928341.0},
     {'Country_Region': 'Albania', 'Population': 2877800.0},
     {'Country_Region': 'Algeria', 'Population': 43851043.0},
     {'Country_Region': 'Andorra', 'Population': 77265.0},
     {'Country_Region': 'Angola', 'Population': 32866268.0},
     {'Country_Region': 'Antigua and Barbuda', 'Population': 97928.0},
     {'Country_Region': 'Argentina', 'Population': 45195777.0},
     {'Country_Region': 'Armenia', 'Population': 2963234.0},
     {'Country_Region': 'Austria', 'Population': 9006400.0},
     {'Country_Region': 'Azerbaijan', 'Population': 10139175.0},
     {'Country_Region': 'Bahamas', 'Population': 393248.0},
     {'Country_Region': 'Bahrain', 'Population': 1701583.0},
     {'Country_Region': 'Bangladesh', 'Population': 164689383.0},
     {'Country_Region': 'Barbados', 'Population': 287371.0},
     {'Country_Region': 'Belarus', 'Population': 9449321.0},
     {'Country_Region': 'Belgium', 'Population': 11589616.0},
     {'Country_Region': 'Belize', 'Population': 397621.0},
     {'Country_Region': 'Benin', 'Population': 12123198.0},
     {'Country_Region': 'Bhutan', 'Population': 771612.0},
     {'Country_Region': 'Bolivia', 'Population': 11673029.0},
     {'Country_Region': 'Bosnia and Herzegovina', 'Population': 3280815.0},
     {'Country_Region': 'Botswana', 'Population': 2351625.0},
     {'Country_Region': 'Brazil', 'Population': 212559409.0},
     {'Country_Region': 'Brunei', 'Population': 437483.0},
     {'Country_Region': 'Bulgaria', 'Population': 6948445.0},
     {'Country_Region': 'Burkina Faso', 'Population': 20903278.0},
     {'Country_Region': 'Burma', 'Population': 54409794.0},
     {'Country_Region': 'Burundi', 'Population': 11890781.0},
     {'Country_Region': 'Cabo Verde', 'Population': 555988.0},
     {'Country_Region': 'Cambodia', 'Population': 16718971.0},
     {'Country_Region': 'Cameroon', 'Population': 26545864.0},
     {'Country_Region': 'Central African Republic', 'Population': 4829764.0},
     {'Country_Region': 'Chad', 'Population': 16425859.0},
     {'Country_Region': 'Chile', 'Population': 19116209.0},
     {'Country_Region': 'Colombia', 'Population': 50882884.0},
     {'Country_Region': 'Congo (Brazzaville)', 'Population': 5518092.0},
     {'Country_Region': 'Congo (Kinshasa)', 'Population': 89561404.0},
     {'Country_Region': 'Comoros', 'Population': 869595.0},
     {'Country_Region': 'Costa Rica', 'Population': 5094114.0},
     {'Country_Region': "Cote d'Ivoire", 'Population': 26378275.0},
     {'Country_Region': 'Croatia', 'Population': 4105268.0},
     {'Country_Region': 'Cuba', 'Population': 11326616.0},
     {'Country_Region': 'Cyprus', 'Population': 1207361.0},
     {'Country_Region': 'Czechia', 'Population': 10708982.0},
     {'Country_Region': 'Denmark', 'Population': 5792203.0},
     {'Country_Region': 'Diamond Princess', 'Population': np.nan},
     {'Country_Region': 'Djibouti', 'Population': 988002.0},
     {'Country_Region': 'Dominica', 'Population': 71991.0},
     {'Country_Region': 'Dominican Republic', 'Population': 10847904.0},
     {'Country_Region': 'Ecuador', 'Population': 17643060.0},
     {'Country_Region': 'Egypt', 'Population': 102334403.0},
     {'Country_Region': 'El Salvador', 'Population': 6486201.0},
     {'Country_Region': 'Equatorial Guinea', 'Population': 1402985.0},
     {'Country_Region': 'Eritrea', 'Population': 3546427.0},
     {'Country_Region': 'Estonia', 'Population': 1326539.0},
     {'Country_Region': 'Eswatini', 'Population': 1160164.0},
     {'Country_Region': 'Ethiopia', 'Population': 114963583.0},
     {'Country_Region': 'Fiji', 'Population': 896444.0},
     {'Country_Region': 'Finland', 'Population': 5540718.0},
     {'Country_Region': 'France', 'Population': 65273512.0},
     {'Country_Region': 'Gabon', 'Population': 2225728.0},
     {'Country_Region': 'Gambia', 'Population': 2416664.0},
     {'Country_Region': 'Georgia', 'Population': 3989175.0},
     {'Country_Region': 'Germany', 'Population': 83783945.0},
     {'Country_Region': 'Ghana', 'Population': 31072945.0},
     {'Country_Region': 'Greece', 'Population': 10423056.0},
     {'Country_Region': 'Grenada', 'Population': 112519.0},
     {'Country_Region': 'Guatemala', 'Population': 17915567.0},
     {'Country_Region': 'Guinea', 'Population': 13132792.0},
     {'Country_Region': 'Guinea-Bissau', 'Population': 1967998.0},
     {'Country_Region': 'Guyana', 'Population': 786559.0},
     {'Country_Region': 'Haiti', 'Population': 11402533.0},
     {'Country_Region': 'Holy See', 'Population': 809.0},
     {'Country_Region': 'Honduras', 'Population': 9904608.0},
     {'Country_Region': 'Hungary', 'Population': 9660350.0},
     {'Country_Region': 'Iceland', 'Population': 341250.0},
     {'Country_Region': 'India', 'Population': 1380004385.0},
     {'Country_Region': 'Indonesia', 'Population': 273523621.0},
     {'Country_Region': 'Iran', 'Population': 83992953.0},
     {'Country_Region': 'Iraq', 'Population': 40222503.0},
     {'Country_Region': 'Ireland', 'Population': 4937796.0},
     {'Country_Region': 'Israel', 'Population': 8655541.0},
     {'Country_Region': 'Italy', 'Population': 60461828.0},
     {'Country_Region': 'Jamaica', 'Population': 2961161.0},
     {'Country_Region': 'Japan', 'Population': 126476458.0},
     {'Country_Region': 'Jordan', 'Population': 10203140.0},
     {'Country_Region': 'Kazakhstan', 'Population': 18776707.0},
     {'Country_Region': 'Kenya', 'Population': 53771300.0},
     {'Country_Region': 'Korea, South', 'Population': 51269183.0},
     {'Country_Region': 'Kosovo', 'Population': 1810366.0},
     {'Country_Region': 'Kuwait', 'Population': 4270563.0},
     {'Country_Region': 'Kyrgyzstan', 'Population': 6524191.0},
     {'Country_Region': 'Laos', 'Population': 7275556.0},
     {'Country_Region': 'Latvia', 'Population': 1886202.0},
     {'Country_Region': 'Lebanon', 'Population': 6825442.0},
     {'Country_Region': 'Lesotho', 'Population': 2142252.0},
     {'Country_Region': 'Liberia', 'Population': 5057677.0},
     {'Country_Region': 'Libya', 'Population': 6871287.0},
     {'Country_Region': 'Liechtenstein', 'Population': 38137.0},
     {'Country_Region': 'Lithuania', 'Population': 2722291.0},
     {'Country_Region': 'Luxembourg', 'Population': 625976.0},
     {'Country_Region': 'Madagascar', 'Population': 27691019.0},
     {'Country_Region': 'Malawi', 'Population': 19129955.0},
     {'Country_Region': 'Malaysia', 'Population': 32365998.0},
     {'Country_Region': 'Maldives', 'Population': 540542.0},
     {'Country_Region': 'Mali', 'Population': 20250834.0},
     {'Country_Region': 'Malta', 'Population': 441539.0},
     {'Country_Region': 'Marshall Islands', 'Population': 58413.0},
     {'Country_Region': 'Mauritania', 'Population': 4649660.0},
     {'Country_Region': 'Mauritius', 'Population': 1271767.0},
     {'Country_Region': 'Mexico', 'Population': 127792286.0},
     {'Country_Region': 'Moldova', 'Population': 4033963.0},
     {'Country_Region': 'Monaco', 'Population': 39244.0},
     {'Country_Region': 'Mongolia', 'Population': 3278292.0},
     {'Country_Region': 'Montenegro', 'Population': 628062.0},
     {'Country_Region': 'Morocco', 'Population': 36910558.0},
     {'Country_Region': 'Mozambique', 'Population': 31255435.0},
     {'Country_Region': 'MS Zaandam', 'Population': np.nan},
     {'Country_Region': 'Namibia', 'Population': 2540916.0},
     {'Country_Region': 'Nepal', 'Population': 29136808.0},
     {'Country_Region': 'Netherlands', 'Population': 17134873.0},
     {'Country_Region': 'New Zealand', 'Population': 4822233.0},
     {'Country_Region': 'Nicaragua', 'Population': 6624554.0},
     {'Country_Region': 'Niger', 'Population': 24206636.0},
     {'Country_Region': 'Nigeria', 'Population': 206139587.0},
     {'Country_Region': 'North Macedonia', 'Population': 2083380.0},
     {'Country_Region': 'Norway', 'Population': 5421242.0},
     {'Country_Region': 'Oman', 'Population': 5106622.0},
     {'Country_Region': 'Pakistan', 'Population': 220892331.0},
     {'Country_Region': 'Panama', 'Population': 4314768.0},
     {'Country_Region': 'Papua New Guinea', 'Population': 8947027.0},
     {'Country_Region': 'Paraguay', 'Population': 7132530.0},
     {'Country_Region': 'Peru', 'Population': 32971846.0},
     {'Country_Region': 'Philippines', 'Population': 109581085.0},
     {'Country_Region': 'Poland', 'Population': 37846605.0},
     {'Country_Region': 'Portugal', 'Population': 10196707.0},
     {'Country_Region': 'Qatar', 'Population': 2881060.0},
     {'Country_Region': 'Romania', 'Population': 19237682.0},
     {'Country_Region': 'Russia', 'Population': 145934460.0},
     {'Country_Region': 'Rwanda', 'Population': 12952209.0},
     {'Country_Region': 'Saint Kitts and Nevis', 'Population': 53192.0},
     {'Country_Region': 'Saint Lucia', 'Population': 183629.0},
     {'Country_Region': 'Saint Vincent and the Grenadines',
      'Population': 110947.0},
     {'Country_Region': 'Samoa', 'Population': 196130.0},
     {'Country_Region': 'San Marino', 'Population': 33938.0},
     {'Country_Region': 'Sao Tome and Principe', 'Population': 219161.0},
     {'Country_Region': 'Saudi Arabia', 'Population': 34813867.0},
     {'Country_Region': 'Senegal', 'Population': 16743930.0},
     {'Country_Region': 'Serbia', 'Population': 8737370.0},
     {'Country_Region': 'Seychelles', 'Population': 98340.0},
     {'Country_Region': 'Sierra Leone', 'Population': 7976985.0},
     {'Country_Region': 'Singapore', 'Population': 5850343.0},
     {'Country_Region': 'Slovakia', 'Population': 5459643.0},
     {'Country_Region': 'Slovenia', 'Population': 2078932.0},
     {'Country_Region': 'Solomon Islands', 'Population': 652858.0},
     {'Country_Region': 'Somalia', 'Population': 15893219.0},
     {'Country_Region': 'South Africa', 'Population': 59308690.0},
     {'Country_Region': 'South Sudan', 'Population': 11193729.0},
     {'Country_Region': 'Spain', 'Population': 46754783.0},
     {'Country_Region': 'Sri Lanka', 'Population': 21413250.0},
     {'Country_Region': 'Sudan', 'Population': 43849269.0},
     {'Country_Region': 'Suriname', 'Population': 586634.0},
     {'Country_Region': 'Sweden', 'Population': 10099270.0},
     {'Country_Region': 'Switzerland', 'Population': 8654618.0},
     {'Country_Region': 'Syria', 'Population': 17500657.0},
     {'Country_Region': 'Taiwan*', 'Population': 23816775.0},
     {'Country_Region': 'Tajikistan', 'Population': 9537642.0},
     {'Country_Region': 'Tanzania', 'Population': 59734213.0},
     {'Country_Region': 'Thailand', 'Population': 69799978.0},
     {'Country_Region': 'Timor-Leste', 'Population': 1318442.0},
     {'Country_Region': 'Togo', 'Population': 8278737.0},
     {'Country_Region': 'Trinidad and Tobago', 'Population': 1399491.0},
     {'Country_Region': 'Tunisia', 'Population': 11818618.0},
     {'Country_Region': 'Turkey', 'Population': 84339067.0},
     {'Country_Region': 'Uganda', 'Population': 45741000.0},
     {'Country_Region': 'Ukraine', 'Population': 43733759.0},
     {'Country_Region': 'United Arab Emirates', 'Population': 9890400.0},
     {'Country_Region': 'United Kingdom', 'Population': 67886004.0},
     {'Country_Region': 'Uruguay', 'Population': 3473727.0},
     {'Country_Region': 'Uzbekistan', 'Population': 33469199.0},
     {'Country_Region': 'Vanuatu', 'Population': 292680.0},
     {'Country_Region': 'Venezuela', 'Population': 28435943.0},
     {'Country_Region': 'Vietnam', 'Population': 97338583.0},
     {'Country_Region': 'West Bank and Gaza', 'Population': 5101416.0},
     {'Country_Region': 'Western Sahara', 'Population': 597330.0},
     {'Country_Region': 'Yemen', 'Population': 29825968.0},
     {'Country_Region': 'Zambia', 'Population': 18383956.0},
     {'Country_Region': 'Zimbabwe', 'Population': 14862927.0},
     {'Country_Region': 'Australia', 'Population': 25459700.0},
     {'Country_Region': 'Canada', 'Population': 37855702.0},
     {'Country_Region': 'China', 'Population': 1404676330.0},
     {'Country_Region': 'US', 'Population': 329466283.0}]
#     create DataFrame df_pop_inc_worldwide from hardcoded 'population_record'
    df_pop_inc_worldwide = pd.DataFrame(population_record)
else:
    print("Using population record (DataFrame df_pop_inc_worldwide) generated by code that processed .csv file related to population record from repository")

In [ ]:
# As long as set of countries in confirmed main is subset of set of countries in population, it is fine.
# To verify this, convert pd.Series of countries to set, & use set functions issubset & symmetric_difference
# If setx.issubset(sety) returns True & setx.symmetric_difference(sety) returns the non-empty set, 
# that non-empty set members are not present in setx

In [ ]:
df_incident_rate = pd.DataFrame()
for i in df_confirmed_ww.columns.tolist():
    df_incident_rate = pd.concat([df_incident_rate, 
                                  df_confirmed_ww[i]*100000/df_pop_inc_worldwide[df_pop_inc_worldwide['Country_Region'].isin([i])]['Population'].values[0]
                                 ], 
                                 axis = 1,
                                 )
df_incident_rate = df_incident_rate.round()

### Core cells to create DataFrames for analysis COMPLETE.

In [ ]:
# Dropdown list options for GUI
countries_list = list(['Worldwide','India', 'US', 'Germany', 'France', \
                    'Italy', 'Canada', 'Japan', 'Israel', 'United Kingdom', \
                    'Russia', 'Brazil', 'Mexico', 'Korea, South', 'Iran', \
                    'Saudi Arabia', 'United Arab Emirates', 'Australia', \
                    'New Zealand','Sweden', 'Switzerland', 'Belgium'])
# countries_list.extend(df_r1.columns.drop_duplicates().tolist()[3:])

countries_list.extend(df_confirmed.columns.drop_duplicates().drop(['India', 'US', 'Germany',
                    'France', 'Italy', 'Canada', 'Japan', 'Israel', 'United Kingdom', \
                    'Russia', 'Brazil', 'Mexico', 'Korea, South', 'Iran', \
                    'Saudi Arabia', 'United Arab Emirates', 'Australia', \
                    'New Zealand', 'Sweden', 'Switzerland', 'Belgium']).tolist())
countries_list.remove('China')

In [ ]:
top10_countries = df_top10_crda.index.tolist()

In [ ]:
trends_list_linear = ['Confirmed', 'Daily Cases', 'Seven Day Moving Average', 'Recovered', 'Deaths', 'Active']
trends_list_log = ['Confirmed_log', 'Daily Cases_log', 'Seven Day Moving Average_log', 'Recovered_log', 'Deaths_log', 'Active_log']

In [ ]:
# hover for linear & log
hv_hover = HoverTool(tooltips = [('Country', '@Variable'), ('Date', '@index{%F}'), ('Cases', '@value{0,}')], 
                     formatters = {'@index': 'datetime'}
                    )

# Creates list of graphs confirmed, daily... Important code, the two list of plots for linear & log can be used in javascript call back..
df_list = [df_confirmed[70:][top10_countries], 
           df_confirmed[70:][top10_countries].diff(1)[df_confirmed[70:][top10_countries].diff(1) > 0], 
           df_confirmed_ww[70:][top10_countries].diff(1).rolling(7).agg('mean').dropna().astype(int),
           df_recovered[70:][top10_countries], df_deaths[70:][top10_countries],
           df_confirmed_ww[70:][top10_countries]- df_recovered_ww[70:][top10_countries] - df_deaths_ww[70:][top10_countries]
          ]

# bokeh plots stored in list linear
hv_list_linear = [hv.render(i.hvplot.line(tools = [hv_hover],
                                          logy = False, width = 1200, height = 600,
                                          grid = True, 
                                          yformatter = NumeralTickFormatter(format = '0.0a')
                                         ).opts(legend_position='right')
                           ) for i in df_list
                 ]

# bokeh plots stored in list log
hv_list_log = [hv.render(i.hvplot.line(tools = [hv_hover],
                                       logy = True, width = 1200, height = 600,
                                       grid = True, ylim = (1, i.max()[0]),
                                       yformatter = NumeralTickFormatter(format = '0.0a')
                                      ).opts(legend_position='right')
                        ) for i in df_list
              ]

In [ ]:
for i, j in zip(trends_list_linear, hv_list_linear):
    j.toolbar.active_drag = None
    j.y_range.start = 0
    j.title.text = i + ' trends for top 10 countries'
    j.legend.title= 'Countries'
    j.ygrid.minor_grid_line_color = 'black'
    j.ygrid.minor_grid_line_alpha = 0.1
    j.toolbar_location = None
    
    j.xaxis.axis_label = 'Date'
    j.xaxis.axis_label_text_font_style = 'bold'
    j.xaxis.major_label_text_font_style = 'bold'
    
    j.yaxis.axis_label = 'COVID19_Case'
    j.yaxis.axis_label_text_font_style = 'bold'
    j.yaxis.major_label_text_font_style = 'bold'

In [ ]:
for i, j in zip(trends_list_log, hv_list_log):
    j.toolbar.active_drag = None
    j.y_range.start = 1
    j.title.text = i + ' trends for top 10 countries'
    j.legend.title= 'Countries'
    j.ygrid.minor_grid_line_color = 'black'
    j.ygrid.minor_grid_line_alpha = 0.1
    j.toolbar_location = None
    
    j.xaxis.axis_label = 'Date'
    j.xaxis.axis_label_text_font_style = 'bold'
    j.xaxis.major_label_text_font_style = 'bold'
    
    j.yaxis.axis_label = 'COVID19_Case'
    j.yaxis.axis_label_text_font_style = 'bold'
    j.yaxis.major_label_text_font_style = 'bold'

In [ ]:
bkh_dict_linear = collections.UserDict(zip(trends_list_linear, hv_list_linear))
bkh_dict_log = collections.UserDict(zip(trends_list_log, hv_list_log))

In [ ]:
df_worldwide_bar = pd.DataFrame(dict(Confirmed = df_confirmed.max().sum(axis = 0),
                                                    Recovered = df_recovered.max().sum(axis = 0),
                                                    Deaths = df_deaths.max().sum(axis = 0),
                                                    Active = df_confirmed.max().sum(axis = 0)
                                                    -df_recovered.max().sum(axis = 0)
                                                    -df_deaths.max().sum(axis = 0)
                                    ), index = ['Worldwide']
                               )

In [ ]:
###################### Trends by country ######################################
# Widgets, selectionbox, checkbox. Flag is kept for R&D purpose.
s200 = pn.widgets.Select(name = 'Trends by Country',
                         options = countries_list, 
                         width = 250
                        )
t198 = Div(text = '<b>Worlwide Trends</b>')

# footer for trends by country
footer_trends_by_country = Div(text = """Graphic: Prasanna Badami 
                                    <div>Source: Center for Systems Science & Engineering, Johns Hopkins University</div> 
                                    <div>Date: {} </div> 
                                    <div style = "font-size:12px"> Note:</div>
                                    <div style = "font-size:12px"><i><t>Incident rate is calculated as confirmed cases per 100,000 population.</i></div>
                                    <div style = "font-size:12px"><i> Case fatality ratio is the ratio of deaths to confirmed cases.</i></div>
                                    <div style = "font-size:12px"><i>United Kingdom has outliers in Recovered Cases data, graphs for certain dates may be not valid for UK.</i></div>""".format(date_on_plot),  
                               width = 1230,
                               background = 'white',
#                                style={'font-family': 'Helvetica', 
#                                       'font-style' : 'italic', 
#                                       'font-size' : '12px', 
#                                       'color': 'black',
#                                       'font-weight': 'bold'},
                              )


# footer for trends comparison(Top 10 countries), footer width 5 less than plot width
footer_trends_comparison = Div(text = """Graphic: Prasanna Badami <div>Source: Center for Systems Science & Engineering, Johns Hopkins University</div> 
                                <div>Date: {} </div>""".format(date_on_plot), 
                               width = 1195,
                               background = 'white',
#                                style={'font-family': 'Helvetica', 
#                                       'font-style' : 'italic', 
#                                       'font-size' : '12px', 
#                                       'color': 'black',
#                                       'font-weight': 'bold'},
                              )

# footer for top 10 bar graph & Worldwide bar graph, footer width 5 less than plot width
footer_top10_ww_bar = Div(text = """Graphic: Prasanna Badami <div>Source: Center for Systems Science & Engineering, Johns Hopkins University</div> 
                                <div>Date: {} </div>""".format(date_on_plot), 
                          width = 925,
                          background = 'white',
#                           style={'font-family': 'Helvetica', 
#                                  'font-style' : 'italic', 
#                                  'font-size' : '12px', 
#                                  'color': 'black',
#                                  'font-weight': 'bold'},
                         )


# HoverTool for trends by country
hover_tool_all = HoverTool(tooltips = [('Date', '@index{%F}'), ('Cases', '@Default{0,}')], 
                           formatters = {'@index': 'datetime'}
                          )

hover_tool_rate = HoverTool(tooltips = [('Date', '@index{%F}'), ('Cases', '@Default'+ ' per 100,000 population')], 
                           formatters = {'@index': 'datetime'}
                           )

hover_tool_ratio = HoverTool(tooltips = [('Date', '@index{%F}'), ('Cases', '@Default' +'%')], 
                           formatters = {'@index': 'datetime'}
                            )

# Figure for graphs, basic things done here.. 
# Confirmed cases
p_confirmed_cases = figure(tools = [hover_tool_all], x_axis_type = 'datetime')
# Daily cases
p_daily_cases = figure(tools = [hover_tool_all], x_axis_type = 'datetime')
# Seven day moving average cases
p_7day_moving_avg = figure(tools = [hover_tool_all], x_axis_type = 'datetime')
# Recovered cases
p_recovered_cases = figure(tools = [hover_tool_all], x_axis_type = 'datetime')
# Deaths
p_deaths = figure(tools = [hover_tool_all], x_axis_type = 'datetime')
# Active cases
p_active = figure(tools = [hover_tool_all], x_axis_type = 'datetime')
# Incident rate
p_incident_rate = figure(tools = [hover_tool_rate], x_axis_type = 'datetime')
# Case fatality ratio
p_case_fatality_ratio = figure(tools = [hover_tool_ratio], x_axis_type = 'datetime')

# Creating column data source files from dataframes for 6 trends:
# 1. Confirmed, 2. Daily, 3. Seven days moving average, 4. Recovered, 5. Deaths & 6. Active
source_c = ColumnDataSource(df_confirmed_ww[70:])
source_c.data['Default'] = source_c.data['Worldwide']
source_daily = ColumnDataSource(df_confirmed_ww[70:].diff(1))
source_daily.data['Default'] = source_daily.data['Worldwide']
source_sda = ColumnDataSource(df_confirmed_ww[70:].diff(1).rolling(7).agg('mean').dropna().astype(int))
source_sda.data['Default'] = source_sda.data['Worldwide']
source_rec = ColumnDataSource(df_recovered_ww[70:])
source_rec.data['Default'] = source_rec.data['Worldwide']
source_dth = ColumnDataSource(df_deaths_ww[70:])
source_dth.data['Default'] = source_dth.data['Worldwide']
source_act = ColumnDataSource(df_confirmed_ww[70:]- df_recovered_ww[70:] - df_deaths_ww[70:])
source_act.data['Default'] = source_act.data['Worldwide']

######################################################
# Incident rate
source_incident_rate = ColumnDataSource(df_incident_rate)
source_incident_rate.data['Default'] = source_incident_rate.data['Worldwide']

# Case fatality ratio
source_case_fatality_ratio = ColumnDataSource(df_deaths_ww[70:]* 100/(df_confirmed_ww[70:]))
source_case_fatality_ratio.data['Default'] = source_case_fatality_ratio.data['Worldwide']

######################################################

# Generating graphs, total 6 graphs for 6 trends. Javascript code controls/changes the 'y' parameter based on user selection.
p_confirmed_cases_GlyphRenderer = p_confirmed_cases.line(source = source_c,
                                                         x = 'index',
                                                         y = 'Default'
                                                        )
p_confirmed_cases.title.text = 'Confirmed Cases'
set_external_graph_prop(p_confirmed_cases)
set_internal_graph_prop(p_confirmed_cases_GlyphRenderer)

p_daily_cases_GlyphRenderer = p_daily_cases.line(source = source_daily,
                                                 x = 'index', 
                                                 y = 'Default'
                                                )
p_daily_cases.title.text = 'Daily Cases'
set_external_graph_prop(p_daily_cases)
set_internal_graph_prop(p_daily_cases_GlyphRenderer)

p_7day_moving_avg_GlyphRenderer = p_7day_moving_avg.line(source = source_sda, 
                                                         x = 'index', 
                                                         y = 'Default'
                                                        )
p_7day_moving_avg.title.text = 'Seven Day Moving Average'
set_external_graph_prop(p_7day_moving_avg)
set_internal_graph_prop(p_7day_moving_avg_GlyphRenderer)

p_recovered_cases_GlyphRenderer = p_recovered_cases.line(source = source_rec, 
                                                         x = 'index', 
                                                         y = 'Default'
                                                        )
p_recovered_cases.title.text = 'Recovered Cases'
set_external_graph_prop(p_recovered_cases)
set_internal_graph_prop(p_recovered_cases_GlyphRenderer)

p_deaths_GlyphRenderer = p_deaths.line(source = source_dth, 
                                       x = 'index', 
                                       y = 'Default'
                                      )
p_deaths.title.text = 'Deaths'
set_external_graph_prop(p_deaths)
set_internal_graph_prop(p_deaths_GlyphRenderer)

p_active_GlyphRenderer = p_active.line(source = source_act,
                                       x = 'index', 
                                       y = 'Default'
                                      )
p_active.title.text = 'Active Cases'
set_external_graph_prop(p_active)
set_internal_graph_prop(p_active_GlyphRenderer)

##############################################################
p_incident_rate.title.text = 'Incident Rate'
prec_r_GlyphRenderer = p_incident_rate.line(source = source_incident_rate,
                                            x = 'index', 
                                            y = 'Default'
                                           )
set_external_graph_prop3(p_incident_rate)
set_internal_graph_prop(prec_r_GlyphRenderer)

p_case_fatality_ratio_GlyphRenderer = p_case_fatality_ratio.line(source = source_case_fatality_ratio,
                                                                 x = 'index', 
                                                                 y = 'Default'
                                                                )
p_case_fatality_ratio.title.text = 'Case Fatality Ratio'
set_external_graph_prop2(p_case_fatality_ratio)
set_internal_graph_prop(p_case_fatality_ratio_GlyphRenderer)
##############################################################


# Arraging 6 graphs in a grid, set merge_tools =False to reflect changes done to toolbar_location
pgrid = gridplot(children = [[p_confirmed_cases, p_daily_cases, p_7day_moving_avg, p_recovered_cases], 
                             [p_deaths, p_active, p_incident_rate, p_case_fatality_ratio]
                            ], 
                 merge_tools = False
                )
# default making grid visible, Javascript code controls/changes visibility based on user interactions with checkbox
pgrid.visible = True
pgrid.background = 'white'
pgrid.name= 'Prasanna Badami'

# JavaScript for select & trends for all countries
callcode = """
    /* alert(c.value) */
    sc.data['Default'] = sc.data[c.value]
    sd.data['Default'] = sd.data[c.value]
    sda.data['Default'] = sda.data[c.value]
    sr.data['Default'] = sr.data[c.value]
    sdt.data['Default'] = sdt.data[c.value]
    sact.data['Default'] = sact.data[c.value]
    
    srec_r.data['Default'] = srec_r.data[c.value]
    sdth_r.data['Default'] = sdth_r.data[c.value]
    
    sc.change.emit()
    sd.change.emit()
    sda.change.emit()
    sr.change.emit()
    sdt.change.emit()
    sact.change.emit()
    
    srec_r.change.emit()
    sdth_r.change.emit()
    pg.visible = true
    tt.text = '<b>' + c.value + ' Trends' + '</b>'
  
"""

# JavaScript 'callcode' is triggered when selection box 'value' changes
s200.jscallback(value = callcode, args = {'sc': source_c,
                                           'sd': source_daily,
                                           'sda': source_sda,
                                           'sr': source_rec,
                                           'sdt': source_dth,
                                           'sact': source_act,
                                           'srec_r': source_incident_rate,
                                           'sdth_r': source_case_fatality_ratio,
                                           'c': s200,
                                           'pg': pgrid,
                                           'tt':t198
                                          }
               )
###############################################################################

###################### Trends comparison(Top 10 countries) ####################
# Widgets
s199 = pn.widgets.Select(name = 'Top 10 Countries Observations', 
                         options = ['Confirmed', 'Daily Cases', 'Seven Day Moving Average', 'Recovered', 'Deaths', 'Active'], 
                         width = 250
                        )
c199 = pn.widgets.Checkbox(name = 'Log Scale', value = False, disabled = False, align = 'end')

@pn.depends(s199, c199)
def get_trendscompare(s199, c199):
    if(c199 == True):
        bkh_dict_r = bkh_dict_log[s199+'_log']
    else:
        bkh_dict_r = bkh_dict_linear[s199]
    return(bkh_dict_r)

###############################################################################

############################# Top 10 bar graph ################################
t200 = Div(text = '<br><br>Top 10 Countries Trends')
# HoverTool for bar graph
hover_tool_bar = HoverTool(tooltips = [('Country', '@x'), ('Cases', '@Cases{0,}')])

# For Bar graph top 10
dft = df_top10_crda.reset_index()
# [('US', 'Confirmed'), ('US', 'Recovered')......]
x = [(i, j) for i in dft['Country/Region'] for j in dft.columns[1:]]
# print(x)

# [cases....] mapped to 'x'
# sum() is used here for concatenation of tuples, resulting in a single tuple
# zip object contains tuples, so 'sum' function second argument is ()
counts = sum(zip(dft.Confirmed, dft.Recovered, dft.Deaths, dft.Active), ())
Cases = list(counts)

source_ww_bar = ColumnDataSource(data = dict(x = x, Cases = Cases))

#  display top 10 data along with plot
table_top10 = hv.Table(dft, kdims = 'Country/Region').opts(width = 600)

# FactorRange is important, else plot will be empty.. setting x_range is very important in Bokeh
pbar = figure(width = 930, 
              height = 600, 
              x_range=FactorRange(*x), 
              tooltips = hover_tool_bar.tooltips, 
              active_drag =None
             )

pbar.vbar(source = source_ww_bar, x = 'x', top = 'Cases',
          width = 1, bottom = 0, 
          fill_color=factor_cmap('x', palette=['royalblue', 'lawngreen', 'red', 'orange'],
                                 factors=['Confirmed', 'Recovered', 'Deaths', 'Active'],
                                 start=1, end=4
                                ), 
          line_color = 'white'
         )
pbar.xaxis.major_label_orientation = 'vertical'
pbar.yaxis.formatter = NumeralTickFormatter(format = '0.0a')
pbar.y_range.start = 0
pbar.background_fill_color = 'white'
pbar.border_fill_color = 'white'
pbar.ygrid.visible  = True
pbar.xgrid.visible  = True
pbar.toolbar_location = None

pbar.xaxis.axis_label = 'Country'
pbar.xaxis.axis_label_text_font_style = 'bold'
# pbar.xaxis.major_label_text_font_style = 'bold'

pbar.yaxis.axis_label = 'COVID19_Case'
pbar.yaxis.axis_label_text_font_style = 'bold'
pbar.yaxis.major_label_text_font_style = 'bold'
###############################################################################

###################### Worldwide bar graph ####################################
t201 = Div(text = '<br><br>Worldwide Trends')
hover_tool_wwbar = HoverTool(tooltips = [('Metric', '@Variable'), ('Cases', '@value{0,}')])
bkh_ww_bar = hv.render(df_worldwide_bar.hvplot.bar(y = ['Active', 'Deaths', 'Recovered', 'Confirmed'],
                                                   tools = [hover_tool_wwbar], 
                                                   color = ['orange', 'red', 'lawngreen', 'royalblue'],
                                                   bar_width = 0.5, width = 930, height = 600,
                                                  ).opts(ylabel = 'COVID-19 Case'
                                                        )
                      )
bkh_ww_bar.yaxis.formatter = NumeralTickFormatter(format = '0.0a')
bkh_ww_bar.toolbar_location = None

bkh_ww_bar.xaxis.axis_label_text_font_style = 'bold'
bkh_ww_bar.xaxis.major_label_text_font_style = 'bold'

bkh_ww_bar.yaxis.axis_label_text_font_style = 'bold'
bkh_ww_bar.yaxis.major_label_text_font_style = 'bold'


# display worldwide data along with plot
table_ww = hv.Table(df_worldwide_bar)
###############################################################################

# Finally arrange everything in order for dashboard.. 
pn_output = pn.Column(pn.Tabs(pn.WidgetBox(s200, t198, 
                               # Javascript is used to control the graphs in bokeh gridplot
                               # Javascript gets activated whenever s200 changes(initiated by user)
                               pgrid,
                               footer_trends_by_country,
                               name = 'Trends by country'
                              ),
                              # widget values as arguments to get_trendscompare
                              pn.WidgetBox(pn.Row(s199, c199),
                                   # Function takes s199/c199 as arguments & is called whenever s199/c199 changes.
                                   # Function definition should have @pn.depends(s199, c199) immediately preceding it.
                                   # It returns graph (bokeh object)
                                   get_trendscompare,
                                   footer_trends_comparison,
                                   name = 'Trends comparison(Top 10 countries)'
                                  ),
                              pn.WidgetBox(pn.Row(pbar, table_top10),footer_top10_ww_bar, name = 'Top 10 bar graph'),
                              pn.WidgetBox(pn.Row(bkh_ww_bar,table_ww), footer_top10_ww_bar, name = 'Worldwide bar graph')
                             )
                     )
# To save to local directory, change to True
if True:
#     To standalone interactive HTML
    pn_output.save('output\COVID19 dashboard-'+ date_on_plot + '.html', 
               resources = INLINE, 
               embed = True, 
               title = 'COVID19 dashboard | ' + date_on_plot
               )

In [ ]:
# for display using mybinder.org
# pn_output

In [ ]:
# To deploy this source code on local system or cloud using Bokeh server
pn_output.servable(title = 'COVID-19_Analysis_dashboard_'+ str(date_on_plot))